# Chain과 복습

In [17]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI() # text-davinch-003 <- llm이라고 칭함

chat = ChatOpenAI() # Chat 에 ChatModel 할당
chat = ChatOpenAI(
    temperature=0.1, # 값이 높을수록 창의적인 답변을 함, (조금 더 인간답다.)
    model='gpt-4' # OpenAI에서 제공하는 ChatModel을 바꿀 수 있다.
    )

In [18]:
chat.predict("hi") # 예측, Chatmodel 에 질의의 결과를 변환


c:\Users\rladm\SSS_langchain\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Hello! How can I assist you today?'

In [19]:
chat.invoke('hi') # 여러 상태, 세부정보를 표현, AIMessage()

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-19e93936-1dd0-4405-b275-fa610b94ca52-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [20]:
# Output Parser

from langchain_core.output_parsers import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
    # strip : 문자열 공백 제거 함수

In [21]:
parser = CommaOutputParser()

parser.parse('Hello, World, hi, How, are, you?')

['Hello', 'World', 'hi', 'How', 'are', 'you?']

In [22]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
# tuple
    ("system", "You are a list Generating machine. Everything you are asked will be ansered with a comma list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

In [23]:
prompt = template.format_messages(
    max_items = 10,
    question = "What are the planets"
)
prompt

[SystemMessage(content='You are a list Generating machine. Everything you are asked will be ansered with a comma list of max 10. Do NOT reply with anything else.'),
 HumanMessage(content='What are the planets')]

In [24]:
result = chat.predict_messages
result

<bound method BaseChatModel.predict_messages of ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019D14767410>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000019D14765B80>, model_name='gpt-4', temperature=0.1, openai_api_key=SecretStr('**********'), openai_proxy='')>

In [25]:
parser = CommaOutputParser()

parser.parse(result.content)

AttributeError: 'function' object has no attribute 'content'

In [26]:
chain = template | chat | CommaOutputParser()

In [27]:
chain.invoke({
    "max_items":10,
    "question" : "What are the planets"
    })

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto',
 'Eris']

In [28]:
template = ChatPromptTemplate.from_messages([
    ('system', 'You are the greatest joker, and you only reply in {language}.'),
    ('ai', 'Hi, my name is {name}'),
    ('human', 'Tell me a joke about the relationship between {blank_a} and {blank_b}. and what is your name?'),
])

In [31]:
chain = template | chat | CommaOutputParser()

In [32]:
chain.invoke({
    "language" : "Korean",
    "name" : "Sungdae",
    "blank_a" : "kR",
    "blank_b" : "JP",
})

['안녕하세요',
 '제 이름은 OpenAI입니다. 한국과 일본의 관계에 대한 농담을 들려드리겠습니다. \n\n한국인이 일본인에게 말했어요. "우리나라에서는 일본 사람들이 만든 게임을 정말 좋아해요." 일본인이 대답했어요. "그래요? 우리나라에서는 한국 드라마를 정말 좋아해요." 그래서 한국인이 말했어요. "그럼 우리나라에서 만든 게임을 보면서 일본 드라마를 봐야겠네요!"']